In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install keras_tqdm

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import tensorflow as tf
from keras.layers import Input, Dense, Lambda, Layer, Activation
from keras.layers.normalization import BatchNormalization
from keras.models import Model
from keras import backend as K
from keras import metrics, optimizers
from keras.callbacks import Callback
import keras

import pydot
import graphviz
from keras.utils import plot_model
from keras_tqdm import TQDMNotebookCallback
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot

import tensorflow as tf
tf.compat.v1.disable_eager_execution()

In [ ]:
!pip install /kaggle/input/iterativestratification/iterative-stratification-master/
!pip install --no-index --find-links /kaggle/input/pytorchtabnet/pytorch_tabnet-2.0.0-py3-none-any.whl pytorch-tabnet

In [ ]:
import numpy as np
import random
import pandas as pd
import matplotlib.pyplot as plt
import os
import copy
import seaborn as sns
import pickle
import sys

from sklearn import preprocessing
from sklearn.metrics import log_loss
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.preprocessing import QuantileTransformer
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss
from pytorch_tabnet.metrics import Metric
from sklearn.metrics import roc_auc_score, log_loss
from sklearn.datasets import make_classification
from sklearn.neighbors import NearestNeighbors
from sklearn.model_selection import StratifiedKFold
from sklearn.decomposition import PCA
from imblearn.over_sampling import SMOTENC

import keras
from keras.layers import Lambda, Input, Dense
from keras.models import Model
from keras.datasets import mnist
from keras.losses import mse, binary_crossentropy
from keras.utils import plot_model
from keras import backend as K
import tensorflow as tf

import warnings
warnings.filterwarnings('ignore')

In [ ]:
import torch
from torch import nn
from torch.utils.data import DataLoader, Dataset
import torch.optim as optim
import torch.nn.functional as F
from torch.optim.lr_scheduler import ReduceLROnPlateau
from pytorch_tabnet.tab_model import TabNetRegressor
from torch.autograd import Variable

os.environ["CUDA_LAUNCH_BLOCKING"] = "1"
from tqdm import tqdm
from sklearn.metrics import log_loss

In [ ]:
from hyperopt import fmin, hp, tpe, Trials, space_eval, STATUS_OK
from hyperopt.pyll import scope as ho_scope
from hyperopt.pyll.stochastic import sample as ho_sample

In [ ]:
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold, MultilabelStratifiedShuffleSplit
from sklearn.metrics import roc_auc_score

In [ ]:

def seed_everything(seed_value):
    random.seed(seed_value)
    np.random.seed(seed_value)
    torch.manual_seed(seed_value)
    os.environ['PYTHONHASHSEED'] = str(seed_value)
    
    if torch.cuda.is_available(): 
        torch.cuda.manual_seed(seed_value)
        torch.cuda.manual_seed_all(seed_value)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False
        
seed_everything(42)

In [ ]:
TRAIN_FEATURES = pd.read_csv('../input/lish-moa/train_features.csv')
TRAIN_TARGETS_SCORED = pd.read_csv('../input/lish-moa/train_targets_scored.csv')
TRAIN_TARGETS_NONSCORED = pd.read_csv('../input/lish-moa/train_targets_nonscored.csv')

TEST_FEATURES = pd.read_csv('../input/lish-moa/test_features.csv')
SAMPLE_SUBMISSION = pd.read_csv('../input/lish-moa/sample_submission.csv')

In [ ]:
TRAIN_FEATURES.head()

In [ ]:
TRAIN_FEATURES.drop(columns=['sig_id'], inplace=True)
TRAIN_TARGETS_SCORED.drop(columns=['sig_id'], inplace=True)
TEST_FEATURES.drop(columns=['sig_id'], inplace=True)

In [ ]:
GENES = [col for col in TRAIN_FEATURES.columns if col.startswith('g-')]
CELLS = [col for col in TRAIN_FEATURES.columns if col.startswith('c-')]

In [ ]:
mean_std = {}
training_features = TRAIN_FEATURES.columns.tolist()
for column in training_features:
    #Skip Categorical Columns
    if column == 'sig_id' or column == 'cp_type' or column == 'cp_dose':
        print('Skip categorical column: ', column)
        continue
    (mu, sigma) = TRAIN_FEATURES[column].mean(), TRAIN_FEATURES[column].std()
    TRAIN_FEATURES[column] = (TRAIN_FEATURES[column]-mu) / sigma
    TEST_FEATURES[column] = (TEST_FEATURES[column] - mu) / sigma
    mean_std[column] = (mu, sigma)

In [ ]:
remove_vehicle = False

if remove_vehicle:
    kept_index = TRAIN_FEATURES['cp_type']=='trt_cp'
    TRAIN_FEATURES = TRAIN_FEATURES.loc[kept_index].reset_index(drop=True)
    train_targets_scored = train_targets_scored.loc[kept_index].reset_index(drop=True)

TRAIN_FEATURES["cp_type"] = (TRAIN_FEATURES["cp_type"]=="trt_cp") + 0
TRAIN_FEATURES["cp_dose"] = (TRAIN_FEATURES["cp_dose"]=="D1") + 0

TEST_FEATURES["cp_type"] = (TEST_FEATURES["cp_type"]=="trt_cp") + 0
TEST_FEATURES["cp_dose"] = (TEST_FEATURES["cp_dose"]=="D1") + 0

In [ ]:
def sampling(args):
    z_mean, z_log_var = args
    epsilon = K.random_normal(shape=tf.shape(z_mean), mean=0., stddev=epsilon_std)
    z = z_mean + K.exp(z_log_var / 2) * epsilon
    return z

In [ ]:
class CustomVariationalLayer(Layer):
    """
    Define a custom layer that learns and performs the training
    This function is borrowed from:
    https://github.com/fchollet/keras/blob/master/examples/variational_autoencoder.py
    """
    def __init__(self, **kwargs):
        self.is_placeholder = True
        super(CustomVariationalLayer, self).__init__(**kwargs)

    def vae_loss(self, x_input, x_decoded, z_log_var_encoded, z_mean_encoded):
        reconstruction_loss = original_dim * metrics.binary_crossentropy(x_input, x_decoded)
        kl_loss = - 0.5 * K.sum(1 + z_log_var_encoded - K.square(z_mean_encoded) - 
                                K.exp(z_log_var_encoded), axis=-1)
        return K.mean(reconstruction_loss + (K.get_value(beta) * kl_loss))

    def call(self, inputs):
        x = inputs[0]
        x_decoded = inputs[1]
        z_log_var_encoded = inputs[2]
        z_mean_encoded = inputs[3]
        loss = self.vae_loss(x, x_decoded, z_log_var_encoded, z_mean_encoded)
        self.add_loss(loss, inputs=inputs)
        # We won't actually use the output.
        return x

In [ ]:
class WarmUpCallback(Callback):
    def __init__(self, beta, kappa):
        self.beta = beta
        self.kappa = kappa
    # Behavior on each epoch
    def on_epoch_end(self, epoch, logs={}):
        if K.get_value(self.beta) <= 1:
            K.set_value(self.beta, K.get_value(self.beta) + self.kappa)

In [ ]:
def custom_dense_layer(x, activation, num_neurons):
    x = Dense(num_neurons, kernel_initializer='glorot_uniform')(x)
    x = BatchNormalization()(x)
    x = Activation(activation)(x)
    return x

def VAE(input_size, num_layers_encoder_mean, neurons_encoder_mean, activation_encoder_mean, num_layers_encoder_variance, neurons_encoder_variance, activation_encoder_variance, num_layers_decoder, neurons_decoder, activation_decoder, latent_dim, x_train, y_train, x_valid, y_valid):
    
    input_vae = Input(shape=(input_size, ))
    
    for i in range(num_layers_encoder_mean - 1):
        if i == 0:
            x = custom_dense_layer(input_vae, activation_encoder_mean, neurons_encoder_mean)
        else:
            x = custom_dense_layer(x, activation_encoder_mean, neurons_encoder_mean)
    
    if num_layers_encoder_mean == 1:
        x = custom_dense_layer(input_vae, activation_encoder_mean, latent_dim)
    else:
        x = custom_dense_layer(x, activation_encoder_mean, latent_dim)
    
    for i in range(num_layers_encoder_variance - 1):
        if i == 0:
            y = custom_dense_layer(input_vae, activation_encoder_variance, neurons_encoder_variance)
        else:
            y = custom_dense_layer(y, activation_encoder_variance, neurons_encoder_variance)
    
    if num_layers_encoder_variance == 1:
        y = custom_dense_layer(input_vae, activation_encoder_variance, latent_dim)
    else:
        y = custom_dense_layer(x, activation_encoder_variance, latent_dim)
            
    output_encoder = Lambda(sampling, output_shape=(latent_dim, ))([x, y])
    
    encoder = Model(input_vae, output_encoder, name='encoder')
    
    for i in range(num_layers_decoder - 1):
        if i == 0:
            z = custom_dense_layer(output_encoder, activation_decoder, neurons_decoder)
        else:
            z = custom_dense_layer(z, activation_decoder, neurons_decoder)
            
    decoder_to_reconstruct = Dense(input_size, kernel_initializer='glorot_uniform', activation='sigmoid')
    
    if num_layers_decoder == 1:
        output_decoder = decoder_to_reconstruct(output_encoder)
    else:
        output_decoder = decoder_to_reconstruct(z)
    
    adam = optimizers.Adam()
    vae_layer = CustomVariationalLayer()([input_vae, output_decoder, y, x])
    vae = Model(input_vae, vae_layer)
    vae.compile(optimizer=adam, loss=None, loss_weights=[beta])
    
    hist = vae.fit(np.array(x_train),
                   shuffle=True,
                   epochs=200,
                   verbose=0,
                   batch_size=50,
                   validation_data=(np.array(x_valid), None),
                   callbacks=[WarmUpCallback(beta, kappa),
                              TQDMNotebookCallback(leave_inner=True, leave_outer=True), 
                              tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                                min_delta=0, patience=5, verbose=0, mode='min',
                                baseline=None, restore_best_weights=True)])
    
    return vae, encoder

In [ ]:
epsilon_std = 1.0
beta = K.variable(0)
kappa = 1
original_dim = 772

In [ ]:
MAX_EPOCH=500
def model_params(n_d=24, n_a=24, n_steps=1, gamma=1.3, lambda_sparse=0):
    tabnet_params = dict(n_d=n_d, n_a=n_a, n_steps=n_steps, gamma=gamma,
                     lambda_sparse=lambda_sparse, optimizer_fn=torch.optim.Adam,
                     optimizer_params=dict(lr=2e-2, weight_decay=1e-5),
                     mask_type='entmax',
                     scheduler_params=dict(mode="min",
                                           patience=5,
                                           min_lr=1e-5,
                                           factor=0.9,),
                     scheduler_fn=torch.optim.lr_scheduler.ReduceLROnPlateau,
                     verbose=10,
                     )
    return tabnet_params

In [ ]:
class LogitsLogLoss(Metric):
    """
    LogLoss with sigmoid applied
    """

    def __init__(self):
        self._name = "logits_ll"
        self._maximize = False

    def __call__(self, y_true, y_pred):
        """
        Compute LogLoss of predictions.

        Parameters
        ----------
        y_true: np.ndarray
            Target matrix or vector
        y_score: np.ndarray
            Score matrix or vector

        Returns
        -------
            float
            LogLoss of predictions vs targets.
        """
        logits = 1 / (1 + np.exp(-y_pred))
        aux = (1-y_true)*np.log(1-logits+1e-15) + y_true*np.log(logits+1e-15)
        return np.mean(-aux)

In [ ]:
def model_train(tabnet_params, TRAIN_FEATURES, TRAIN_TARGETS_SCORED, NB_SPLITS=2, optimization=True):
    mskf = MultilabelStratifiedKFold(n_splits=NB_SPLITS, random_state=0, shuffle=True)
    oof_preds = []
    oof_targets = []
    scores = []
    scores_auc = []
    for fold_nb, (train_idx, val_idx) in enumerate(mskf.split(TRAIN_FEATURES, TRAIN_TARGETS_SCORED)):
        print("FOLDS : ", fold_nb)
        ## model
        X_train, y_train = TRAIN_FEATURES.values[train_idx, :], TRAIN_TARGETS_SCORED.values[train_idx, :]
        X_val, y_val = TRAIN_FEATURES.values[val_idx, :], TRAIN_TARGETS_SCORED.values[val_idx, :]
        model = TabNetRegressor(**tabnet_params)
        
        model.fit(X_train=X_train,
                    y_train=y_train,
                    eval_set=[(X_val, y_val)],
                    eval_name = ["val"],
                    eval_metric = ["logits_ll"],
                    max_epochs=MAX_EPOCH,
                    patience=10, batch_size=1024, virtual_batch_size=128,
                    num_workers=0, drop_last=False,
                    loss_fn=torch.nn.functional.binary_cross_entropy_with_logits);

        preds_val = model.predict(X_val)
        # Apply sigmoid to the predictions
        preds =  1 / (1 + np.exp(-preds_val))
        score = np.min(model.history["val_logits_ll"])
        oof_preds.append(preds_val)
        oof_targets.append(y_val)
        scores.append(score)
        if optimization:
            return scores
        return model

In [ ]:
space = [hp.quniform('num_layers_encoder_mean', 1, 3, 1), 
    hp.quniform('neurons_encoder_mean', 256, 2048, 32), 
    hp.choice('activation_encoder_mean', ('relu', 'selu', 'sigmoid', 'tanh', 'elu')), 
    hp.quniform('num_layers_encoder_variance', 1, 3, 1), 
    hp.quniform('neurons_encoder_variance', 256, 2048, 32), 
    hp.choice('activation_encoder_variance', ('relu', 'selu', 'sigmoid', 'tanh', 'elu')), 
    hp.quniform('num_layers_decoder', 1, 3, 1),
    hp.quniform('neurons_decoder', 256, 2048, 32), 
    hp.choice('activation_decoder', ('relu', 'selu', 'sigmoid', 'tanh', 'elu')),
    hp.quniform('latent_dim', 400, 770, 10),
    hp.quniform('n_d', 8, 64, 2), 
    hp.quniform('n_steps', 1, 10, 1), 
    hp.uniform('gamma', 1, 2), 
    hp.uniform('lambda_sparse', 0, 0.01), 
    hp.quniform('n_components', 40, 100, 5),
    hp.choice('optimization', ['True']),  
    hp.choice('TRAINING_FEATURES', [TRAIN_FEATURES]), 
    hp.choice('TRAIN_TARGETS_SCORED', [TRAIN_TARGETS_SCORED])
]

In [ ]:
def final_model_creator(args):
    
    num_layers_encoder_mean, neurons_encoder_mean, activation_encoder_mean, num_layers_encoder_variance, neurons_encoder_variance, activation_encoder_variance, num_layers_decoder, neurons_decoder, activation_decoder, latent_dim, n_d, n_steps, gamma, lambda_sparse, n_components, optimization, TRAIN_FEATURES, TRAIN_TARGETS_SCORED = args
    
    GENES = [col for col in TRAIN_FEATURES.columns if col.startswith('g-')]
    CELL = [col for col in TRAIN_FEATURES.columns if col.startswith('c-')]
    
    # Build Gene Encoder
    TRAIN_GENE = TRAIN_FEATURES[GENES]
    X_train, X_test, y_train, y_test = train_test_split(TRAIN_GENE, TRAIN_TARGETS_SCORED, 
                                                    test_size=0.1, random_state=42)
    gene_vae, gene_encoder = VAE(772, int(num_layers_encoder_mean), int(neurons_encoder_mean), 
                                 activation_encoder_mean, int(num_layers_encoder_variance), int(neurons_encoder_variance), 
                                 activation_encoder_variance, int(num_layers_decoder), int(neurons_decoder), 
                                 activation_decoder, int(latent_dim), X_train, y_train, X_test, y_test)
    
    # Do PCA on Cell Viability
    TRAIN_CELL = TRAIN_FEATURES[CELL]
    pca = PCA(n_components=int(n_components))
    
    # Compressing Table
    TRAIN_GENE = pd.DataFrame(gene_encoder.predict(TRAIN_GENE))
    TRAIN_CELL = pca.fit_transform(TRAIN_CELL)
    TRAIN_CELL = pd.DataFrame(data=TRAIN_CELL)
    
    #Column names
    GENES = GENES[:int(latent_dim)]
    CELL = CELL[:int(n_components)]
    REST = TRAIN_FEATURES.iloc[:, :3].columns
    
    # Compressing to one table
    TRAIN_FEATURES = pd.concat([TRAIN_FEATURES.iloc[:, :3], TRAIN_GENE, TRAIN_CELL], axis=1)
    TRAIN_FEATURES.columns = list(REST) + GENES + CELL
    
    #Applying RankGauss for normailization
    for col in (GENES + CELL):

        transformer = QuantileTransformer(n_quantiles=100,random_state=0, output_distribution="normal")
        vec_len = len(TRAIN_FEATURES[col].values)
        vec_len_test = len(TEST_FEATURES[col].values)
        raw_vec = TRAIN_FEATURES[col].values.reshape(vec_len, 1)
        transformer.fit(raw_vec)

        TRAIN_FEATURES[col] = transformer.transform(raw_vec).reshape(1, vec_len)[0]
    
    # Building, compiling and training the model
    tabnet_params = model_params(n_d=int(n_d), n_a=int(n_d), n_steps=int(n_steps), gamma=gamma, lambda_sparse=lambda_sparse)
    sc_model = model_train(tabnet_params, NB_SPLITS=10, optimization=optimization, TRAIN_FEATURES=TRAIN_FEATURES, TRAIN_TARGETS_SCORED=TRAIN_TARGETS_SCORED)
    
    # return score
    if optimization:
        return sc_model[0]
    return gene_encoder, sc_model

In [ ]:
#trials = Trials()

In [ ]:
%cd '../input/moa-prediction-ba'
trials = pickle.load(open("trial_150.hyperopt", "rb"))
%cd ..
%cd ..
%cd working

In [ ]:
evals = 150
while evals < 170:
    best = fmin(fn=final_model_creator, space=space, algo=tpe.suggest, max_evals=evals+10, trials=trials)
    with open('trial_{}'.format(str(evals+10)) + ".hyperopt", "wb") as f:
        pickle.dump(trials, f)
    evals += 10